In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# === Load German Credit dataset ===
column_names = [
    'Status_checking_account', 'Duration', 'Credit_history', 'Purpose', 'Credit_amount',
    'Savings_account_bonds', 'Employment', 'Installment_rate', 'Personal_status_and_sex',
    'Other_debtors', 'Residence_since', 'Property', 'Age', 'Other_installment_plans',
    'Housing', 'Number_credits', 'Job', 'People_liable', 'Telephone', 'Foreign_worker', 'Risk'
]

df = pd.read_csv("german.data", sep=' ', header=None, names=column_names)
df['target'] = df['Risk'].map({1: 0, 2: 1})
df.drop(columns=['Risk'], inplace=True)

# === Encode categoricals ===
categorical_cols = [
    'Status_checking_account', 'Credit_history', 'Purpose', 'Savings_account_bonds',
    'Employment', 'Personal_status_and_sex', 'Other_debtors', 'Property',
    'Other_installment_plans', 'Housing', 'Job', 'Telephone', 'Foreign_worker'
]
numerical_cols = [col for col in df.columns if col not in categorical_cols + ['target']]

for col in categorical_cols:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

# === Normalize numeric features ===
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# === Train-test split ===
X = df.drop(columns=['target']).values.astype(np.float32)
y = df['target'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# === Reshape for BDLSTM (samples, timesteps=1, features=20) ===
X_train_seq = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_seq = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))


In [28]:
!pip install optuna


In [29]:
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import tensorflow as tf

def objective(trial):
    # Suggest hyperparameters
    units = trial.suggest_categorical('units', [32, 64, 128])
    dropout = trial.suggest_float('dropout', 0.2, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'rmsprop'])
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])

    # Optimizer
    optimizer = Adam(learning_rate=learning_rate) if optimizer_name == 'adam' else RMSprop(learning_rate=learning_rate)

    # Build model
    model = Sequential([
        Bidirectional(LSTM(units, return_sequences=True), input_shape=(1, X_train.shape[1])),
        Dropout(dropout),
        Bidirectional(LSTM(units, return_sequences=True)),
        Dropout(dropout),
        GlobalMaxPooling1D(),
        Dense(units, activation='relu'),
        Dropout(dropout),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Fit model
    model.fit(X_train_seq, y_train, epochs=20, batch_size=batch_size, validation_split=0.2, verbose=0)

    # Predict and evaluate
    y_pred_prob = model.predict(X_test_seq).ravel()
    y_pred = (y_pred_prob > 0.64).astype(int)
    return f1_score(y_test, y_pred)


In [30]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)


[I 2025-04-13 16:37:36,963] A new study created in memory with name: no-name-aec23921-f1f2-423a-a9be-01813eef4290
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step


[I 2025-04-13 16:37:51,819] Trial 0 finished with value: 0.4367816091954023 and parameters: {'units': 64, 'dropout': 0.3395480811980938, 'learning_rate': 0.001043793891851771, 'optimizer': 'adam', 'batch_size': 16}. Best is trial 0 with value: 0.4367816091954023.
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step


[I 2025-04-13 16:38:02,257] Trial 1 finished with value: 0.4948453608247423 and parameters: {'units': 32, 'dropout': 0.2068935690147701, 'learning_rate': 0.002184427666002941, 'optimizer': 'adam', 'batch_size': 32}. Best is trial 1 with value: 0.4948453608247423.
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

[I 2025-04-13 16:42:04,181] Trial 17 finished with value: 0.5384615384615384 and parameters: {'units': 64, 'dropout': 0.23731304490451333, 'learning_rate': 0.003118102011611157, 'optimizer': 'adam', 'batch_size': 16}. Best is trial 7 with value: 0.5871559633027523.
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 304ms/step


[I 2025-04-13 16:42:23,233] Trial 18 finished with value: 0.36585365853658536 and parameters: {'units': 32, 'dropout': 0.3674546160899827, 'learning_rate': 0.002280709183477372, 'optimizer': 'rmsprop', 'batch_size': 32}. Best is trial 7 with value: 0.5871559633027523.
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step


[I 2025-04-13 16:42:35,481] Trial 19 finished with value: 0.6115702479338843 and parameters: {'units': 128, 'dropout': 0.22795038426019668, 'learning_rate': 0.0071402157467890075, 'optimizer': 'adam', 'batch_size': 64}. Best is trial 19 with value: 0.6115702479338843.


In [31]:
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)


Best hyperparameters: {'units': 128, 'dropout': 0.22795038426019668, 'learning_rate': 0.0071402157467890075, 'optimizer': 'adam', 'batch_size': 64}
Best F1 Score: 0.6115702479338843


In [32]:
y_pred_prob = model.predict(X_test_seq).ravel()
y_pred = (y_pred_prob > 0.64).astype(int)

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("📈 F1 Score:", f1_score(y_test, y_pred))
print("🏆 ROC AUC:", roc_auc_score(y_test, y_pred_prob))
print("\n📄 Classification Report:\n", classification_report(y_test, y_pred))

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
✅ Accuracy: 0.755
📈 F1 Score: 0.4367816091954023
🏆 ROC AUC: 0.7885714285714286

📄 Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.94      0.84       140
           1       0.70      0.32      0.44        60

    accuracy                           0.76       200
   macro avg       0.73      0.63      0.64       200
weighted avg       0.75      0.76      0.72       200

